## Daten und nötigen Packages importieren
## Und mit scatter Plot Daten visualisieren

In [ ]:
import pandas as pd 
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures
import seaborn as sns
from scipy.optimize import curve_fit

df = pd.read_csv("[Dateipfad]")
# mit dtype = np.[datatype] kann man die Daten gleich beim Import anpassen zB. ("...", dtype = np.float)
# Um Spalten vorab zu entfernen:
# df = df.drop("Spaltenname", axis = 1)

sns.pairplot(df)

## Transformieren der X-Werte
## Aufteilen Trainings und Testdaten

In [ ]:
# achte auf die doppelten eckigen Klammern damit Datentyp ein Dataframe bleibt
# Oder .values dahinter -> df wird zu np.array
# wird benötigt für die K-Fold Cross Validation

X = df[['X1-Werte', 'X2-Werte']].values
# Oder: Die Spalte "xyz" und "abc" soll nicht zur Vorhersage verwendet werden:
# X = df.drop("xyz", axis = 1).drop("abc", axis = 1).values
y = df[['Y-Werte']].values

# X-Werte transformieren
# polynominale Terme erstellen (zusätzliche Spalten hinzufügen - Potenzen)
# X1², X1*X2 und X2² werden hinzugefügt (zusätzlich zu X1 und X2)
pf = PolynomialFeatures(degree = 4, include_bias = False)
pf.fit(X)
# bei 2. Grad, Spalten = 5
# bei 3. Grad, Spalten = 9
# bei 4. Grad, Spalten = 14
# pf.power_ zeigt Form an
X_transformed = pf.transform(X)[:, [0, 1, 2, 3, 4]]
# schauen welche Spalten zu einem besseren Ergebnis führen oder welche Spalten 
# egals sind

X_train_trans, X_test_trans, y_train, y_test = train_test_split(X_transformed, y) 
# default = 1/4 testdaten

## Trainiere das Modell mit den Trainingsdaten
## Teste das Modell mit den Testdaten

In [ ]:
# und an LineareRegression weitergeben
model = LinearRegression()
model.fit(X_train_trans, y_train)

# model.predict speichert meine dazugehörigen/berechneten Y-Werte in der angegebenen Variabel
ytestprdct = model.predict(X_test_transformed)

## Andere Möglichkeit (Prof. Vogl)

In [ ]:
# definiere die Funktion, die an die Daten angepaßt werden soll
def func(x, a, b, c, d):
    return a+b*x+c*x**2+d*x**3
# führe den Fit durch!
# ausgegeben werden die Fit-Parameter und die Kovarianzmatrix der Parameter
popt, pcov = curve_fit(func, X_train,y_train)
print(popt) # optimalen Parameter
print(pcov) # Kovarianzmatrix

Xtrainprdct = func(X_train,*popt) 
Xtestprdct = func(X_test,*popt)

## R² - Bestimmtheitsmaß um Modell zu evaluieren

R² := 1 - Summe(Fehler des Modells)/Summe(Fehler des Durchschnitts)

1: heißt beschreibt Daten perfekt

0: heißt Daten werden genauso gut durch Durchschnitt beschrieben

In [ ]:
model.score(X_test_trans, y_test)

## K-Fold Cross Validation

mehrere Modelle auf verschiedene Teile der Daten trainieren 

-> Mittelung der verschiedenen Bestimmtheitsmaße

In [ ]:
kf = KFold(n_splits = , 4, shuffle = True)

# kurze Schreibweise:
scores = cross_val_score(LinearRegression(), X, y, cv = kf)
# weiß nicht ob das so mit Poly. Regr. funktioniert..
print(np.mean(scores))

# lange Schreibweise:
kf.split(X)
# generiert Indizes um die Daten zu splitten
# mit diesen Indizes kann man dann auf die X und Y Werte zugreifen

## Good to know 
### Geradengleichung

f(x) = m * x + t

y = coef * x + intercept

In [ ]:
model.intercept_
model.coef_
# damit kann jetzt die Regressionsgerade gezeichnet werden

### Ebenengleichung

f(x1, x2) = a * x1 + b* x2 + c

y = coef1 * x1 + coef2 * x2 + intercept

In [ ]:
# Achtung bei mehreren Variablen darauf acten, dass die Variablen/Daten 
# unkorreliert sind! 